# Evaluate Multi Agent System

This notebook evaluates the accuracy and performance of the three-agent system of banks made in this repository with real world data.

## Evaluation with initial economic variables only

This section tests the multi-agent system pipeline with just the initialization of the economic variables. In this case, the agents have no prior information on their behavior in previous quarters. This evaluation thus relies solely on the LLM's training data to make good decisions and properly propagate changes in economic variables.

Hypothesis: the system will not match actual proceedings as the bank agents make the best decisions on all the information it knows (which is vastly more in some cases than some of the case studies' banks would have had access to) and the economic agent simply works off of these banks being representative of other similar banks.

In [ ]:
# Import GDP from US Bureau of Economic Analysis
import os

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("BEA_API_KEY")
api_url = "https://apps.bea.gov/api/data/"
params = {
    "UserID": api_key,
    "method": "GetData",
    "DataSetName": "NIPA",
    "TableName": "T10101", # Table with GDP Growth Rates
    "Frequency": "Q",
    "Year": "ALL",
    "ResultFormat": "JSON"  # Use XML if preferred
}

# Fetch data
response = requests.get(api_url, params=params)
response.raise_for_status()  # Raise an exception for HTTP errors
data = response.json()

# Extract data nodes
results = data["BEAAPI"]["Results"]["Data"]

# Transform into a DataFrame
df = pd.DataFrame(results)

# Optionally, convert relevant columns to numeric for analysis
df["DataValue"] = pd.to_numeric(df["DataValue"], errors="coerce")
df["TimePeriod"] = pd.to_datetime(df["TimePeriod"])

# Select relevant columns
df = df[["TimePeriod", "DataValue"]]

# Rename columns for clarity
df.rename(columns={"TimePeriod": "Quarter", "DataValue": "GDP Growth Rate"}, inplace=True)

# Set index
df.set_index("Quarter", inplace=True)

print(df.head())